## Project 3: RNN

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


### Step 1: Import text

In [2]:
text = open('input.txt',encoding='utf8').read().lower()
print('Text Length:', len(text))

Text Length: 1115394


#### Basic configurations

In [3]:
chars = sorted(list(set(text)))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# Cut the text into small sentences with fixed length
txtlen = 100
step = 10
sentences = []
next_chars = []
for i in range(0, len(text) - txtlen, step):
    sentences.append(text[i: i + txtlen])
    next_chars.append(text[i + txtlen])

print('Total Chars: ', len(chars))
print('Total Sentences: ', len(sentences))

Total Chars:  39
Total Sentences:  111530


### Step 2: Vectorization

In [4]:
x_train = np.zeros((len(sentences), txtlen, len(chars)), dtype=np.bool)
y_train = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_train[i, t, char_indices[char]] = 1
    y_train[i, char_indices[next_chars[i]]] = 1

### Step 3: Setup the model

In [5]:
model = Sequential()

# LSTM
model.add(LSTM(128, input_shape=(txtlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

rms = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               86016     
_________________________________________________________________
dense_1 (Dense)              (None, 39)                5031      
_________________________________________________________________
activation_1 (Activation)    (None, 39)                0         
Total params: 91,047
Trainable params: 91,047
Non-trainable params: 0
_________________________________________________________________


### Step 4: Let's make some predictions

In [6]:
def sample(preds, diversity=1.0):
    preds = np.asarray(preds).astype('float64')
    log_preds = np.log(preds) / diversity
    exp_preds = np.exp(log_preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
batch_size = 1024
epochs = 1

diversity = 1.0

for i in range(1,51):
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
    start_point = random.randint(0, len(text) - txtlen - 1)
    
    if i%5 == 0:
        
        generated = ''
        sentence = text[start_point : start_point + txtlen]
        generated += sentence
        print('\n---------- Generating with: ----------')
        print(generated)
        print('\n---------- Prediction: ----------')
        
        for i in range(300):
            x_pred = np.zeros((1, txtlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        
        print()
        
model.save('model.h5')

Epoch 1/1
111530/111530 [==============================] - 15s 139us/step - loss: 2.5997
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 2.0306
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.8343
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.7170
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.6353

---------- Generating with: ----------
:
a match!

adrian:
though this island seem to be desert,--

sebastian:
ha, ha, ha! so, you're paid.

---------- Prediction: ----------


nirsto:
gat i sadeby for in that laicls rowns;
all my donemas; and i bestechisk.
and was bring of mistive hist mus it forle
thes all gavely toy, say here the trupins'
tage mest cbosing, that,
as ho their king, now wnom
of cloop scorned band brood the catible.

bostonga:
doth apor lidy light; on th
Epoch 1/1
111530/111530 [==============================] - 13s 120us/step - lo

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


rk! what the disgived so look of changed
of soin be instory pards breaken and tymel.
when icabe! well-twex your honour venge; steen concerves
than my good titr toolly an lew a
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.1277
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.1233
Epoch 1/1
111530/111530 [==============================] - 13s 118us/step - loss: 1.1223
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.1168
Epoch 1/1
111530/111530 [==============================] - 13s 119us/step - loss: 1.1138

---------- Generating with: ----------
ding to your love,
and little look'd for at your helping hands.

king richard ii:
alack, why am i se

---------- Prediction: ----------
n, breck acquuesion abolest
stayely had an'er, not in this death.

are togome:
me to agaves but unto thines. your dead.

blurnes:
then his! live must shoind richard.

second citize:
ah, looks! where's di